# Проект Дашборд конверсий

In [105]:
import pandas as pd
import requests
import datetime as dt
pd.options.mode.chained_assignment = None  # default='warn'

## Получаем данные о регистрациях и посещениях по API

In [106]:
req_visits = requests.get('https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01')
df_visits = pd.DataFrame(req_visits.json())

req_registrations = requests.get('https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01')
df_registrations = pd.DataFrame(req_registrations.json())

## Агрегируем данные о регистрациях и посещениях и сохраняем полученный датафрейм в conversion.json

In [107]:
#Группируем данные о посещениях

##Группируем по visit_id, берем только последнее посещение
df_visits['datetime'] = pd.to_datetime(df_visits['datetime'])
df_visits_last = df_visits.loc[df_visits.groupby('visit_id')['datetime'].idxmax()]

##Отфильтровываем посещения ботов
df_visits_filtered = df_visits_last[~df_visits_last['user_agent'].str.contains('bot', case=False)]

##Группируем посещения по дате и платформе
df_visits_filtered['datetime'] = df_visits_filtered['datetime'].dt.strftime('%Y-%m-%d')
df_visits_grouped = df_visits_filtered.groupby(['datetime', 'platform']).size().reset_index(name='visits')

##Переименовываем столбцы согласно заданию
df_visits_grouped.columns = ['date_group', 'platform', 'visits']

#Группируем данные о регистрациях
##Группируем регистрации по дате и платформе
df_registrations['date_group'] = pd.to_datetime(df_registrations['datetime']).dt.strftime('%Y-%m-%d')
df_registrations_grouped = df_registrations.groupby(['date_group', 'platform']).size().reset_index(name='registrations')

#Объединяем датасэты
df_conversions = pd.merge(df_visits_grouped, df_registrations_grouped, on=['date_group', 'platform'], how='outer')

#Считаем конверсию из посещений в регистрацию
df_conversions['conversion'] = (df_conversions['registrations'] * 100 / df_conversions['visits']).round(2)

#Сортируем строки согласно заданию
df_conversions = df_conversions.sort_values(by='date_group')

#Сохраняем полученный датафрейм в формате JSON
df_conversions.to_json('conversion.json')

## Получаем данные по рекламным кампаниям

In [108]:
df_ads = pd.read_csv('https://drive.google.com/uc?id=12vCtGhJlcK_CBcs8ES3BfEPbk6OJ45Qj')

## Агрегируем данные по рекламным кампаниям и сохраняем полученный датафрейм в ads.json

In [115]:
#Группируем рекламу дате и компании, считаем стоимость
df_ads['date_group'] = pd.to_datetime(df_ads['date']).dt.strftime('%Y-%m-%d')
df_ads_grouped = df_ads.groupby(['date_group']).agg({'cost': 'sum', 'utm_campaign': 'first'}).reset_index()

#Объединяем полученный датафрейм с df_conversions
df_ads_result = pd.merge(df_conversions, df_ads_grouped, on='date_group', how='left')

#Заполняем пустые значения в датафрейме
df_ads_result['cost'] = df_ads_result['cost'].fillna(0)
df_ads_result['utm_campaign'] = df_ads_result['utm_campaign'].fillna('none')

#Сортируем строки согласно заданию
df_ads_result = df_ads_result.sort_values(by='date_group')

#Сохраняем полученный датафрейм в формате JSON
df_ads_result.to_json('ads.json')